In [ ]:
"""
Purpose: 
To create the function that will create a mapping correspondence
of skeletal branches (those with no node degrees higher than 3)
and the meshes themselves




"""

In [1]:
from os import sys
sys.path.append("../../meshAfterParty/")

In [2]:
import skeleton_utils as sk
import trimesh_utils as tu
import trimesh
import numpy_utils as nu
import trimesh_utils as xu
import numpy as np


In [3]:
saved_file = "../../Platinum_Compartment/soma_touching_mesh_data.npz"
current_mesh_data = nu.load_dict(saved_file)

In [4]:
#all of the branches
current_mesh_data

{0: {'branch_meshes': [<trimesh.Trimesh(vertices.shape=(153651, 3), faces.shape=(308343, 3))>,
   <trimesh.Trimesh(vertices.shape=(87663, 3), faces.shape=(176175, 3))>,
   <trimesh.Trimesh(vertices.shape=(56972, 3), faces.shape=(114671, 3))>,
   <trimesh.Trimesh(vertices.shape=(54953, 3), faces.shape=(110120, 3))>,
   <trimesh.Trimesh(vertices.shape=(53735, 3), faces.shape=(107788, 3))>,
   <trimesh.Trimesh(vertices.shape=(51875, 3), faces.shape=(104099, 3))>,
   <trimesh.Trimesh(vertices.shape=(29324, 3), faces.shape=(58779, 3))>,
   <trimesh.Trimesh(vertices.shape=(29151, 3), faces.shape=(58543, 3))>,
   <trimesh.Trimesh(vertices.shape=(14725, 3), faces.shape=(29598, 3))>,
   <trimesh.Trimesh(vertices.shape=(3865, 3), faces.shape=(7718, 3))>,
   <trimesh.Trimesh(vertices.shape=(1159, 3), faces.shape=(2282, 3))>],
  'soma_meshes': array([<trimesh.Trimesh(vertices.shape=(14158, 3), faces.shape=(28309, 3))>,
         <trimesh.Trimesh(vertices.shape=(20481, 3), faces.shape=(40659, 3))>],

In [5]:
# visualize all of the branches and the meshes
sk.graph_skeleton_and_mesh(other_meshes=list(current_mesh_data[0]["branch_meshes"]) + list(current_mesh_data[0]["soma_meshes"]),
                          other_meshes_colors="random",
                           other_skeletons = current_mesh_data[0]["branch_skeletons"],
                          other_skeletons_colors="random")

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [8]:
all_branches = current_mesh_data[0]["branch_meshes"] 
all_skeletons = current_mesh_data[0]["branch_skeletons"] 
branch_index = 0
curr_mesh_to_clean = all_branches[branch_index]
curr_skeleton_to_clean = all_skeletons[branch_index]
cleaned_skeleton = sk.clean_skeleton(curr_skeleton_to_clean,
                   distance_func=sk.skeletal_distance,
                  min_distance_to_junction = 4001,
                  return_skeleton=True,
                  print_flag=False)

sk.graph_skeleton_and_mesh(other_meshes=[curr_mesh_to_clean],
                          other_skeletons=[cleaned_skeleton])


Using Distance measure skeletal_distance


In [17]:
# want to clean all of the branches
sk = reload(sk)
skelton_cleaning_threshold = 4001
current_mesh_data[0]["branch_skeletons_cleaned"] = [sk.clean_skeleton(curr_skeleton_to_clean,
                   distance_func=sk.skeletal_distance,
                  min_distance_to_junction = skelton_cleaning_threshold,
                  return_skeleton=True,
                  print_flag=False) for curr_skeleton_to_clean in current_mesh_data[0]["branch_skeletons"]]


Using Distance measure skeletal_distance



Using Distance measure skeletal_distance



Using Distance measure skeletal_distance



Using Distance measure skeletal_distance



Using Distance measure skeletal_distance


...


Using Distance measure skeletal_distance



Using Distance measure skeletal_distance



Using Distance measure skeletal_distance



Using Distance measure skeletal_distance



Using Distance measure skeletal_distance
no small end nodes to get rid of so returning whole skeleton


In [18]:
# visualize all of the branches and the meshes
sk.graph_skeleton_and_mesh(other_meshes=list(current_mesh_data[0]["branch_meshes"]) + list(current_mesh_data[0]["soma_meshes"]),
                          other_meshes_colors="random",
                           other_skeletons = current_mesh_data[0]["branch_skeletons_cleaned"],
                          other_skeletons_colors="random")

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [ ]:
from skeleton_utils import *
def mesh_correspondence_vp1(main_mesh,edges,
                                 buffer=0.01,
                                bbox_ratio=1.2,
                                 distance_threshold=2000,
                             significance_threshold=500,
                                print_flag=False):
    """
    Purpose: Will return significant mesh pieces that are
    not already accounteed for by the skeleton
    
    Example of how to run
    
    main_mesh_path = Path("./Dustin/Dustin.off")
    main_mesh = trimesh.load_mesh(str(main_mesh_path.absolute()))
    skeleton_path = main_mesh_path.parents[0] / Path(main_mesh_path.stem + "_skeleton.cgal")
    edges = sk.read_skeleton_edges_coordinates(str(skeleton_path.absolute()))

    # turn this into nodes and edges
    main_mesh_nodes, main_mesh_edges = sk.read_skeleton_verts_edges(str(skeleton_path.absolute()))
    sk.graph_skeleton_and_mesh(
                main_mesh_verts=main_mesh.vertices,
                main_mesh_faces=main_mesh.faces,
                unique_skeleton_verts_final = main_mesh_nodes,
                edges_final=main_mesh_edges,
                buffer = 0
                              )
                              
    leftover_pieces =  mesh_subtraction_by_skeleton(main_mesh,edges,
                                 buffer=0.01,
                                bbox_ratio=1.2,
                                 distance_threshold=500,
                             significance_threshold=500,
                                print_flag=False)
                                
    # Visualize the results: 
    pieces_mesh = trimesh.Trimesh(vertices=np.array([]),
                                 faces=np.array([]))

    for l in leftover_pieces:
        pieces_mesh += l

    sk.graph_skeleton_and_mesh(
                main_mesh_verts=pieces_mesh.vertices,
                main_mesh_faces=pieces_mesh.faces,
                unique_skeleton_verts_final = main_mesh_nodes,
                edges_final=main_mesh_edges,
                buffer = 0
                              )
    
    """
    

    
    """ This part will be removed for trial because don't trust bbox restriction without whole skeleton
    
    skeleton_nodes = edges.reshape(-1,3)
    skeleton_bounding_corners = np.vstack([np.max(skeleton_nodes,axis=0),
               np.min(skeleton_nodes,axis=0)])
    
        
    main_mesh_bbox_restricted, faces_bbox_inclusion = tu.bbox_mesh_restriction(main_mesh,
                                                                        skeleton_bounding_corners,
                                                                        bbox_ratio)
    
    if type(main_mesh_bbox_restricted) == type(trimesh.Trimesh()):
        print(f"Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = {len(main_mesh_bbox_restricted.faces)}")
    else:
        print("***** Bounding Box Restricted Mesh is empty ****")
        main_mesh_bbox_restricted = main_mesh
        faces_bbox_inclusion = np.arange(0,len(main_mesh.faces))
    
    """
    
    main_mesh_bbox_restricted = main_mesh
    faces_bbox_inclusion = np.arange(0,len(main_mesh.faces))

    
    start_time = time.time()

    #face_subtract_color = []
    face_subtract_indices = []

    
    edge_loop_print=False
    for i,ex_edge in tqdm(enumerate(edges)):
        #print("\n------ New loop ------")
        #print(ex_edge)
        
        # ----------- creating edge and checking distance ----- #
        loop_start = time.time()
        
        edge_line = ex_edge[1] - ex_edge[0]
        sum_threshold = 0.001
        if np.sum(np.abs(edge_line)) < sum_threshold:
            if edge_loop_print:
                print(f"edge number {i}, {ex_edge}: has sum less than {sum_threshold} so skipping")
            continue
#         if edge_loop_print:
#             print(f"Checking Edge Distance = {time.time()-loop_start}")
#         loop_start = time.time()
        
        cob_edge = change_basis_matrix(edge_line)
        
#         if edge_loop_print:
#             print(f"Change of Basis Matrix calculation = {time.time()-loop_start}")
#         loop_start - time.time()
        
        #get the limits of the example edge itself that should be cutoff
        edge_trans = (cob_edge@ex_edge.T)
        slice_range = np.sort(edge_trans[2,:])

        # adding the buffer to the slice range
        slice_range_buffer = slice_range + np.array([-buffer,buffer])
        
#         if edge_loop_print:
#             print(f"Calculate slice= {time.time()-loop_start}")
#         loop_start = time.time()

        # generate face midpoints from the triangles
        #face_midpoints = np.mean(main_mesh_bbox_restricted.vertices[main_mesh_bbox_restricted.faces],axis=1) # Old way
        face_midpoints = main_mesh_bbox_restricted.triangles_center
        
#         if edge_loop_print:
#             print(f"Face midpoints= {time.time()-loop_start}")
#         loop_start = time.time()
        
        #get the face midpoints that fall within the slice (by lookig at the z component)
        fac_midpoints_trans = cob_edge@face_midpoints.T
        
#         if edge_loop_print:
#             print(f"Face midpoints transform= {time.time()-loop_start}")
#         loop_start = time.time()
        
        
        
#         if edge_loop_print:
#             print(f"edge midpoint= {time.time()-loop_start}")
#         loop_start = time.time()
        
        slice_mask_pre_distance = ((fac_midpoints_trans[2,:]>slice_range_buffer[0]) & 
                      (fac_midpoints_trans[2,:]<slice_range_buffer[1]))

        if edge_loop_print:
            print(f"Applying slice restriction = {time.time()-loop_start}")
        loop_start = time.time()
        
#         print(f"(fac_midpoints_trans.T)[:,:2] = {(fac_midpoints_trans.T)[:,:2].shape}")
#         print(f"face_midpoints[:,:2] = {face_midpoints[:,:2].shape}")
#         print(f"edge_trans.T = {(edge_trans.T).shape}")
#         print(f"ex_edge = {ex_edge.shape}")
        """ 6/18 change
        # apply the distance threshold to the slice mask
        
        edge_midpoint = np.mean(ex_edge,axis=0)
        #raise Exception("Add in part for distance threshold here")
        distance_check = np.linalg.norm(face_midpoints[:,:2] - edge_midpoint[:2],axis=1) < distance_threshold
        
        
        """
        edge_midpoint = np.mean(edge_trans.T,axis=0)
        distance_check = np.linalg.norm((fac_midpoints_trans.T)[:,:2] - edge_midpoint[:2],axis=1) < distance_threshold
        
        

        slice_mask = slice_mask_pre_distance & distance_check
        
        if edge_loop_print:
            print(f"Applying distance restriction= {time.time()-loop_start}")
        loop_start = time.time()


        face_list = np.arange(0,len(main_mesh_bbox_restricted.faces))[slice_mask]

        #get the submesh of valid faces within the slice
        if len(face_list)>0:
            main_mesh_sub = main_mesh_bbox_restricted.submesh([face_list],append=True)
        else:
            main_mesh_sub = []
        
        

        if type(main_mesh_sub) != type(trimesh.Trimesh()):
            if edge_loop_print:
                print(f"THERE WERE NO FACES THAT FIT THE DISTANCE ({distance_threshold}) and Z transform requirements")
                print("So just skipping this edge")
            continue

        if edge_loop_print:
            print(f"getting submesh= {time.time()-loop_start}")
        loop_start = time.time()
        
        #get all disconnected mesh pieces of the submesh and the face indices for lookup later
        sub_components,sub_components_face_indexes = tu.split(main_mesh_sub,only_watertight=False)
        if type(sub_components) != type(np.array([])) and type(sub_components) != list:
            print(f"meshes = {sub_components}, with type = {type(sub_components)}")
            if type(sub_components) == type(trimesh.Trimesh()) :
                sub_components = [sub_components]
            else:
                raise Exception("The sub_components were not an array, list or trimesh")
        
        if edge_loop_print:
            print(f"splitting the mesh= {time.time()-loop_start}")
        loop_start = time.time()

        #getting the indices of the submeshes whose bounding box contain the edge 
        """ 6-19: might want to use bounding_box_oriented? BUT THIS CHANGE COULD SLOW IT DOWN
        contains_points_results = np.array([s_comp.bounding_box_oriented.contains(ex_edge.reshape(-1,3)) for s_comp in sub_components])
        """
        contains_points_results = np.array([s_comp.bounding_box.contains(ex_edge.reshape(-1,3)) for s_comp in sub_components])
        
        containing_indices = (np.arange(0,len(sub_components)))[np.sum(contains_points_results,axis=1) >= len(ex_edge)]
        
        if edge_loop_print:
            print(f"containing indices= {time.time()-loop_start}")
        loop_start = time.time()

        if len(containing_indices) != 1: 
            if edge_loop_print:
                print(f"--> Not exactly one containing mesh: {containing_indices}")
            if len(containing_indices) > 1:
                sub_components_inner = sub_components[containing_indices]
                sub_components_face_indexes_inner = sub_components_face_indexes[containing_indices]
            else:
                sub_components_inner = sub_components
                sub_components_face_indexes_inner = sub_components_face_indexes

            #get the center of the edge
            edge_center = np.mean(ex_edge,axis=0)
            #print(f"edge_center = {edge_center}")

            #find the distance between eacch bbox center and the edge center
            bbox_centers = [np.mean(k.bounds,axis=0) for k in sub_components_inner]
            #print(f"bbox_centers = {bbox_centers}")
            closest_bbox = np.argmin([np.linalg.norm(edge_center-b_center) for b_center in bbox_centers])
            #print(f"bbox_distance = {closest_bbox}")
            edge_skeleton_faces = faces_bbox_inclusion[face_list[sub_components_face_indexes_inner[closest_bbox]]]
            if edge_loop_print:
                print(f"finding closest box when 0 or 2 or more containing boxes= {time.time()-loop_start}")
            loop_start = time.time()
        else:# when only one viable submesh piece and just using that sole index
            #print(f"only one viable submesh piece so using index only number in: {containing_indices}")
            
            edge_skeleton_faces = faces_bbox_inclusion[face_list[sub_components_face_indexes[containing_indices[0]]]]
            if edge_loop_print:
                print(f"only 1 containig face getting the edge_skeleton_faces= {time.time()-loop_start}")
            loop_start = time.time()

        if len(edge_skeleton_faces) < 0:
            print(f"****** Warning the edge index {i}: had no faces in the edge_skeleton_faces*******")
        face_subtract_indices.append(edge_skeleton_faces)
        if edge_loop_print:
                print(f"check and append for face= {time.time()-loop_start}")
        #face_subtract_color.append(viable_colors[i%len(viable_colors)])
        
    print(f"Total Mesh subtraction time = {np.round(time.time() - start_time,4)}")
    #print(f"face_subtract_indices = {face_subtract_indices}")
    
    unique_removed_faces = []
    if len(face_subtract_indices)>0:
        all_removed_faces = np.concatenate(face_subtract_indices)

        unique_removed_faces = set(all_removed_faces)

        faces_to_keep = set(np.arange(0,len(main_mesh.faces))).difference(unique_removed_faces)
        new_submesh = main_mesh.submesh([list(faces_to_keep)],only_watertight=False,append=True)
    else:
        new_submesh = main_mesh
    
    return unique_removed_faces
    """    
    significant_pieces = split_significant_pieces(new_submesh,
                                                         significance_threshold,
                                                         print_flag=False)
    return significant_pieces
    """

    

# -- decomposing all of limbs into branches and finding mesh correspondence using mesh_correspondence_vp1 -- 

In [ ]:
sk = reload(sk)
tu = reload(tu)
from tqdm.notebook import tqdm
import time

start_time = time.time()

limb_correspondence = dict()
soma_containing_idx= 0

total_branch_meshes = []
total_branch_skeletons = []

for soma_containing_idx in current_mesh_data.keys():
    for limb_idx,curr_limb_mesh in enumerate(current_mesh_data[soma_containing_idx]["branch_meshes"]):
        print(f"Working on limb #{limb_idx}")
        limb_correspondence[limb_idx] = dict()
        curr_limb_sk = current_mesh_data[soma_containing_idx]["branch_skeletons"][limb_idx]
        curr_limb_branches_sk = sk.decompose_skeleton_to_branches(curr_limb_sk) #the line that is decomposing to branches
        for j,curr_branch_sk in tqdm(enumerate(curr_limb_branches_sk)):
            limb_correspondence[limb_idx][j] = dict()
            curr_branch_face_correspondence = mesh_correspondence_vp1(
                                    curr_limb_mesh,
                                    curr_branch_sk,
                                    buffer=100,
                                    bbox_ratio=1.2,
                                    distance_threshold=3000,
                                    significance_threshold=500,
                                    print_flag=False
                                )
            if len(curr_branch_face_correspondence) > 0:
                curr_submesh = curr_limb_mesh.submesh([list(curr_branch_face_correspondence)],append=True)
            else:
                curr_submesh = trimesh.Trimesh(vertices=np.array([]),faces=np.array([]))
                
            limb_correspondence[limb_idx][j]["correspondence_mesh"] = curr_submesh
            limb_correspondence[limb_idx][j]["correspondence_face_idx"] = curr_branch_face_correspondence
            
            total_branch_meshes.append(curr_submesh)
            total_branch_skeletons.append(curr_branch_sk)
            
print(f"Total time = {time.time() - start_time}")

# -- Visulaizing all of branches and the mesh correspondence from version 1

In [ ]:
sk = reload(sk)
import matplotlib_utils as mu
mu = reload(mu)
sk.graph_skeleton_and_mesh(other_meshes=total_branch_meshes,
                          other_meshes_colors="random",
                           other_skeletons=total_branch_skeletons,
                           other_skeletons_colors="random"
                          )


# !!! Now starting the methods to make the mesh subtraction more exact !!

In [ ]:
def get_skeletal_distance(main_mesh,edges,
                                 buffer=0.01,
                                bbox_ratio=1.2,
                               distance_threshold=3000,
                                print_flag=False):
    """
    Purpose: To return the histogram of distances along a mesh subtraction process
    so that we could evenutally find an adaptive distance threshold
    
    
    """

    
    main_mesh_bbox_restricted = main_mesh
    faces_bbox_inclusion = np.arange(0,len(main_mesh.faces))

    
    start_time = time.time()
    face_subtract_indices = []
    
    edge_loop_print=True
    total_distances = []
    total_distances_std = []
    for i,ex_edge in tqdm(enumerate(edges)):
        #print("\n------ New loop ------")
        #print(ex_edge)
        
        # ----------- creating edge and checking distance ----- #
        loop_start = time.time()
        
        edge_line = ex_edge[1] - ex_edge[0]
        sum_threshold = 0.001
        if np.sum(np.abs(edge_line)) < sum_threshold:
            if edge_loop_print:
                print(f"edge number {i}, {ex_edge}: has sum less than {sum_threshold} so skipping")
            continue
        
        cob_edge = change_basis_matrix(edge_line)
        
        #get the limits of the example edge itself that should be cutoff
        edge_trans = (cob_edge@ex_edge.T)
        #slice_range = np.sort((cob_edge@ex_edge.T)[2,:])
        slice_range = np.sort(edge_trans[2,:])

        # adding the buffer to the slice range
        slice_range_buffer = slice_range + np.array([-buffer,buffer])

        # generate face midpoints from the triangles
        #face_midpoints = np.mean(main_mesh_bbox_restricted.vertices[main_mesh_bbox_restricted.faces],axis=1) # Old way
        face_midpoints = main_mesh_bbox_restricted.triangles_center
        
        
        #get the face midpoints that fall within the slice (by lookig at the z component)
        fac_midpoints_trans = cob_edge@face_midpoints.T
        
        slice_mask_pre_distance = ((fac_midpoints_trans[2,:]>slice_range_buffer[0]) & 
                      (fac_midpoints_trans[2,:]<slice_range_buffer[1]))

        edge_midpoint = np.mean(edge_trans.T,axis=0)
        distance_check = np.linalg.norm((fac_midpoints_trans.T)[:,:2] - edge_midpoint[:2],axis=1) < distance_threshold
        
        slice_mask = slice_mask_pre_distance & distance_check
        
        face_list = np.arange(0,len(main_mesh_bbox_restricted.faces))[slice_mask]

        #get the submesh of valid faces within the slice
        if len(face_list)>0:
            main_mesh_sub = main_mesh_bbox_restricted.submesh([face_list],append=True)
        else:
            main_mesh_sub = []

        if type(main_mesh_sub) != type(trimesh.Trimesh()):
            if edge_loop_print:
                print(f"THERE WERE NO FACES THAT FIT THE DISTANCE ({distance_threshold}) and Z transform requirements")
                print("So just skipping this edge")
            continue


        #get all disconnected mesh pieces of the submesh and the face indices for lookup later
        sub_components,sub_components_face_indexes = tu.split(main_mesh_sub,only_watertight=False)
        if type(sub_components) != type(np.array([])) and type(sub_components) != list:
            #print(f"meshes = {sub_components}, with type = {type(sub_components)}")
            if type(sub_components) == type(trimesh.Trimesh()) :
                sub_components = [sub_components]
            else:
                raise Exception("The sub_components were not an array, list or trimesh")
        

        #getting the indices of the submeshes whose bounding box contain the edge 
        contains_points_results = np.array([s_comp.bounding_box.contains(ex_edge.reshape(-1,3)) for s_comp in sub_components])
        
        containing_indices = (np.arange(0,len(sub_components)))[np.sum(contains_points_results,axis=1) >= len(ex_edge)]
        
        if len(containing_indices) != 1: 
            if edge_loop_print:
                print(f"--> Not exactly one containing mesh: {containing_indices}")
            if len(containing_indices) > 1:
                sub_components_inner = sub_components[containing_indices]
                sub_components_face_indexes_inner = sub_components_face_indexes[containing_indices]
            else:
                sub_components_inner = sub_components
                sub_components_face_indexes_inner = sub_components_face_indexes

            #get the center of the edge
            edge_center = np.mean(ex_edge,axis=0)
            #print(f"edge_center = {edge_center}")

            #find the distance between eacch bbox center and the edge center
            bbox_centers = [np.mean(k.bounds,axis=0) for k in sub_components_inner]
            #print(f"bbox_centers = {bbox_centers}")
            closest_bbox = np.argmin([np.linalg.norm(edge_center-b_center) for b_center in bbox_centers])
            #print(f"bbox_distance = {closest_bbox}")
            edge_skeleton_faces = faces_bbox_inclusion[face_list[sub_components_face_indexes_inner[closest_bbox]]]
            

        else:# when only one viable submesh piece and just using that sole index
            edge_skeleton_faces = faces_bbox_inclusion[face_list[sub_components_face_indexes[containing_indices[0]]]]


        if len(edge_skeleton_faces) < 0:
            print(f"****** Warning the edge index {i}: had no faces in the edge_skeleton_faces*******")
        face_subtract_indices.append(edge_skeleton_faces)
        
        
        #---- calculating the relevant distances ---- #
        
        face_midpoints = (main_mesh_bbox_restricted.triangles_center)[edge_skeleton_faces]
        #print(f"edge_skeleton_faces.shape = {edge_skeleton_faces.shape}")
        fac_midpoints_trans = cob_edge@face_midpoints.T
        #print(f"fac_midpoints_trans.shape = {fac_midpoints_trans.shape}")
        mesh_slice_distances = np.linalg.norm((fac_midpoints_trans.T)[:,:2] - edge_midpoint[:2],axis=1)
        #print(f"mesh_slice_distances.shape = {mesh_slice_distances.shape}")
        
        total_distances.append(np.mean(mesh_slice_distances))
        total_distances_std.append(np.std(mesh_slice_distances))
    
    
    
    
    if len(face_subtract_indices)>0:
        all_removed_faces = np.concatenate(face_subtract_indices)

        unique_removed_faces = set(all_removed_faces)

        #faces_to_keep = set(np.arange(0,len(main_mesh.faces))).difference(unique_removed_faces)
        new_submesh = main_mesh.submesh([list(unique_removed_faces)],only_watertight=False,append=True)
        new_submesh = sk.split_significant_pieces(new_submesh,significance_threshold=1)[0]
    
    else:
        new_submesh = trimesh.Trimesh()
 
    
    
    return total_distances,total_distances_std,new_submesh


def mesh_correspondence_adaptive_distance(curr_branch_skeleton,
                                          curr_branch_mesh,
                                         skeleton_segment_width = 1000)

    #making the skeleton smaller
    
    new_skeleton  = sk.resize_skeleton_branch(curr_branch_skeleton,segment_width = skeleton_segment_width)

    segment_skeletal_mean_distances,segment_skeletal_std_distances,mesh_correspondence = get_skeletal_distance(
                        main_mesh = curr_branch_mesh,
                        edges = new_skeleton,
                        buffer=100,
                        bbox_ratio=1.2,
                        distance_threshold=3000,
                        print_flag=False
    )
        
    #now use the new submesh to calculate the new threshold
    # -- Step where I compute the new threshold and and then rerun it -- #
    if len(segment_skeletal_mean_distances) > 4:
        filtered_measurements = np.array(segment_skeletal_mean_distances[1:-1])
        filtered_measurements_std = np.array(segment_skeletal_std_distances[1:-1])
    else:
        filtered_measurements = np.array(segment_skeletal_mean_distances)
        filtered_measurements_std = np.array(segment_skeletal_std_distances)



    #filter out the other outliers: do anything higher than 150% of median should be discounted
    median_value = np.median(filtered_measurements)
    outlier_mask = filtered_measurements < median_value*1.5
    filtered_measurements = filtered_measurements[outlier_mask]
    filtered_measurements_std = filtered_measurements_std[outlier_mask]

    print(f"filtered_measurements = {filtered_measurements}")

    np.mean(filtered_measurements),np.std(filtered_measurements)

    # try the mesh subtraction again 
    buffer = 100

    total_threshold = np.max(filtered_measurements) + 2*np.max(filtered_measurements_std)
    print(f"new_threshold = {total_threshold}")
    segment_skeletal_mean_distances_2,filtered_measurements_std,mesh_correspondence_2 = get_skeletal_distance(
                        main_mesh = mesh_correspondence,
                        edges = new_skeleton,
                        buffer=100,
                        bbox_ratio=1.2,
                        distance_threshold=total_threshold,
                        print_flag=False
    )
    
    return mesh_correspondence_2




In [ ]:
sk = reload(sk)

#branch_idx = 6 #looked better
#branch_idx = 3 #works well now after debugging the resize branch
#branch_idx = 2 #this one is really small and has skewed distribution (but should get rid os these)
#branch_idx = 1
#branch_idx = 4
branch_idx = 5
branch_idx = 3
branch_idx = 1



#curr_limb_mesh = current_mesh_data[0]["branch_meshes"][0]
curr_branch_skeleton = total_branch_skeletons[branch_idx]
curr_branch_mesh = total_branch_meshes[branch_idx]

#making the skeleton smaller
skeleton_segment_width = 1000
new_skeleton  = sk.resize_skeleton_branch(curr_branch_skeleton,segment_width = skeleton_segment_width)

segment_skeletal_mean_distances,segment_skeletal_std_distances,mesh_correspondence = get_skeletal_distance(
                    main_mesh = curr_branch_mesh,
                    edges = new_skeleton,
                    buffer=100,
                    bbox_ratio=1.2,
                    distance_threshold=3000,
                    print_flag=False
)


sk.graph_skeleton_and_mesh(other_meshes=[mesh_correspondence],
                              other_meshes_colors=["random"],
                              other_skeletons=[new_skeleton])


from matplotlib import pyplot as plt

total_distance_checks = segment_skeletal_mean_distances

total_averages = []

start_perc = 0.30
end_perc = 0.70
total_distance_indexes = list(range(len(total_distance_checks)))
middle_distances = total_distance_checks[int(len(total_distance_checks)*start_perc):int(len(total_distance_checks)*end_perc)]
middle_distances_indexes = total_distance_indexes[int(len(total_distance_checks)*start_perc):int(len(total_distance_checks)*end_perc)]
total_averages.append(np.mean(middle_distances))
print(f"average middle_distances = {total_averages[-1]}")
fig,ax = plt.subplots(1,2)
ax[0].plot(total_distance_indexes,total_distance_checks,c="pink")
ax[0].plot(middle_distances_indexes,middle_distances,c="black")
ax[1].plot(total_distance_indexes,segment_skeletal_std_distances)
plt.title(f"branch {branch_idx} with average = {total_averages[-1]}"
         f"\n From {start_perc} to {end_perc}")
plt.show()



In [ ]:
# -- Step where I compute the new threshold and and then rerun it -- #
if len(segment_skeletal_mean_distances) > 4:
    filtered_measurements = np.array(segment_skeletal_mean_distances[1:-1])
    filtered_measurements_std = np.array(segment_skeletal_std_distances[1:-1])
else:
    filtered_measurements = np.array(segment_skeletal_mean_distances)
    filtered_measurements_std = np.array(segment_skeletal_std_distances)



#filter out the other outliers: do anything higher than 150% of median should be discounted
median_value = np.median(filtered_measurements)
outlier_mask = filtered_measurements < median_value*1.5
filtered_measurements = filtered_measurements[outlier_mask]
filtered_measurements_std = filtered_measurements_std[outlier_mask]

print(f"filtered_measurements = {filtered_measurements}")

np.mean(filtered_measurements),np.std(filtered_measurements)

# try the mesh subtraction again 
buffer = 100

total_threshold = np.max(filtered_measurements) + 2*np.max(filtered_measurements_std)
print(f"new_threshold = {total_threshold}")
segment_skeletal_mean_distances_2,filtered_measurements_std,mesh_correspondence_2 = get_skeletal_distance(
                    main_mesh = mesh_correspondence,
                    edges = new_skeleton,
                    buffer=100,
                    bbox_ratio=1.2,
                    distance_threshold=total_threshold,
                    print_flag=False
)


#remove any measurements

In [ ]:


sk.graph_skeleton_and_mesh(other_meshes=[mesh_correspondence_2,mesh_correspondence],
                              other_meshes_colors=["random"],
                              other_skeletons=[new_skeleton])
